<div class="alert alert-info">
<font size="4", color = "black"><b>Исследование рынка заведений общественного питания Москвы. Исследование категории "кофейни".</b></font>
Для  анализа выполним следующие этапы:

1) Изучим исходный датасет; \
2) Найдем столбцы с пропусками, попробуем их проанализировать, при необходимости немного очистить данные; \
3) После предобработки проведем исследовательский анализ и попробуем оценить рынок кафе и других заведений общественного питания в Москве, выделим некоторые особенности, визуализируем основные показатели;\
4) Проанализируем рынок кофеен Москвы, предложим идеи для открытия этого бизнеса;\
5) Сделаем выводы о проделанной работе

## 1 Загрузка данных и изучение общей информации

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from plotly import graph_objects as go

In [2]:
!pip install missingno

In [3]:
import missingno as msno

In [4]:
import seaborn as sns


In [ ]:
try:
    data=pd.read_csv(r"C:\Users\Владислав\Documents\dat_an\moscow_places.csv")
except:
    data=pd.read_csv('/datasets/moscow_places.csv')

#### 1.1 Загрузите данные и изучите общую информацию

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
msno.matrix(data, labels=True)

Одни данные представлены полность, другие лишь в незначительном количестве, что может затруднить анализ.
При этом заменять отсутсвующие данные на средние или медианные в данном случае не корректно, поскольку
зависимость может быть от многих факторов и незаполненных данных в некоторых столбцах больше, чем заполненных

### 2 Предобработка данных

#### 2.1 Удаление дубликатов

In [ ]:
data=data.drop_duplicates()

In [ ]:
data['name'].nunique()

In [ ]:
data['address'].nunique()

Нельзя однозначно сделать вывод о неявных дубликатах, так как в одном месте может находиться
несколько заведений(фудкорт), а названия могут повторяться или различаться по регистру. Попробуем
удалить дубликаты по совокупности признаков (адрес и название).

In [ ]:
data.shape

In [ ]:

data=data.drop_duplicates(['name','address'])

In [ ]:
data.shape

In [ ]:

data['name']=data['name'].str.lower()
data['category']=data['category'].str.lower()
data['address']=data['address'].str.lower()
data=data.drop_duplicates(['name', 'category', 'address'])

<div class="alert alert-info">
Таким образом, было найдено два неявных дубликата

In [ ]:
data.shape

Удалим наиболее пустые данные, не содержащие основных показателей для дальнейшего анализа

In [ ]:
data.info()

Данные очищены, оставшиеся дубликаты сложно найти без дополнительной информации


#### 2.2 Создайте столбец street с названиями улиц из столбца с адресом.

In [ ]:
data['address']

In [ ]:
data['street']=data['address'].apply(lambda x:x.split(',')[1].strip())

In [ ]:
data.loc[data['street'] == 'проспект мира','name'].count()

#### 2.3 Создайте столбец is_24/7 с обозначением, что заведение работает ежедневно и круглосуточно (24/7)

In [ ]:
def workinghours(data):
    hours=data['hours']
    if data['hours']=='ежедневно, круглосуточно':
        return True
    else:
        return False
        
    

In [ ]:
data['is_24/7']=data.apply(workinghours,axis=1)

<div class="alert alert-info">
В рамках первого этапа были добавлены новые столбцы, с названием улицы и информацией, работает ли заведение круглосуточно,
а также были удалены некоторые неявные дубликаты

### 3 Анализ данных

### 3.1 
Какие категории заведений представлены в данных?
Исследуйте количество объектов общественного питания по категориям: рестораны, кофейни, пиццерии,
    бары и так далее. Постройте визуализации. Ответьте на вопрос о распределении заведений по 
    категориям.

In [ ]:
data['category'].unique()

In [ ]:
catcount=data.pivot_table(index='category',values='address',aggfunc='count')
catcount.columns=['quantity']
catcount['percent']=catcount['quantity']/(catcount['quantity'].sum())*100
catcount=catcount.sort_values(by='quantity',ascending=False).reset_index()
catcount

In [ ]:
catcount['quantity'].sum()

### исследуем количество заведений и мест в них, в зависимости от категории

In [ ]:
with plt.style.context('seaborn'):
    plt.bar(catcount['category'],catcount['quantity'])
    plt.xticks(rotation=45)
    plt.title('Количество заведений в зависимости от категории')
    plt.ylabel('Количество заведений')
    plt.xlabel('Категории заведений')
    plt.show()


In [ ]:
with plt.style.context('seaborn'):
    plt.bar(catcount['category'],catcount['percent'])
    plt.xticks(rotation=45)
    plt.title('Процент заведений в зависимости от категории')
    plt.ylabel('Процент заведений данной категории, %')
    plt.xlabel('Категории заведений')
    plt.show()


Самая популярная категория - кафе, что не удивительно, на втором месте находятся рестораны

#### 3.2
Исследуйте количество посадочных мест в местах по категориям: рестораны, кофейни, пиццерии, бары и так далее. Постройте 
визуализации. Проанализируйте результаты и сделайте выводы


In [ ]:
seatcount=data.pivot_table(index='category',values='seats',aggfunc=['mean','median'])
seatcount.columns=['mean_seats','median_seats']
seatcount=seatcount.sort_values(by='mean_seats',ascending=False).reset_index()
seatcount

Таким образом, больше всего мест в барах и ресторанах

In [ ]:
with plt.style.context('seaborn'):
    plt.bar(seatcount['category'],seatcount['mean_seats'])
    plt.xticks(rotation=45)
    plt.title('Среднее количество мест в зависимости от категории')
    plt.ylabel('среднее количество мест')
    plt.xlabel('Категории заведений')
    plt.show()

In [ ]:
seatcount=seatcount.sort_values(by='median_seats',ascending=False).reset_index()

In [ ]:
with plt.style.context('seaborn'):
    plt.bar(seatcount['category'],seatcount['median_seats'])
    plt.xticks(rotation=45)
    plt.title(' Медианное количество мест в зависимости от категории')
    plt.ylabel('медианное количество мест')
    plt.xlabel('Категории заведений')
    plt.show()

В данном случае, лучше пользоваться медианным значением. Таким образом, больше всего мест в ресторанах и барах

#### 3.3 Рассмотрите и изобразите соотношение сетевых и несетевых заведений в датасете. Каких заведений больше?

In [ ]:
data.head()

In [ ]:
chain=data['chain'].value_counts()
chain

In [ ]:
print(f'сетевых заведений: {chain[1]}')
print(f'не сетевых заведений: {chain[0]}')

In [ ]:
fig2=go.Figure(data=[go.Pie(labels=['сетевое','не сетевое'],values=[chain[1],chain[0]])])
fig2.update_layout(title='Процентное соотношение сетевых и несетевых заведений ')
fig2.show()

Таким образом, сетевых заведений заметно меньше

##### 3.3.1 Какие категории заведений чаще являются сетевыми? Исследуйте данные и ответьте на вопрос графиком.

In [ ]:

chaincat=data.pivot_table(index='category',columns='chain',values='name',aggfunc=['count'])
chaincat.columns=['non-chain','chain']
chaincat=chaincat.reset_index()
chaincat

In [ ]:
chaincat['total']=(chaincat['chain']+chaincat['non-chain'])
chaincat['percent']=(chaincat['chain']/chaincat['total'])*100
chaincat=chaincat.sort_values(by='percent',ascending=False)

In [ ]:
display(chaincat)

In [ ]:
#процентное соотношение сетевых по категориям
fig3=go.Figure(data=[go.Pie(labels=chaincat['category'],values=chaincat['percent'])])
fig3.update_layout(title='Процентное соотношение сетевых заведений по категориям')
fig3.show()

Больше всего сетевых среди булочных, пиццерий и кофеен. Это можно было предположить, зная крупные сетевые заведения 

In [ ]:
import plotly.express as px
fig4=px.bar(chaincat,x='category',y='percent',color='total')
fig4.update_layout(title='Процент сетевых по категориях'
                  ,xaxis_title=' категории',yaxis_title='процент сетевых заведений')
fig4.show()
#,text=chaincat['ratio'].apply(lambda x:'{0:1.2f}%'.format(x))

#### 3.4
Сгруппируйте данные по названиям заведений и найдите топ-15 популярных сетей в Москве. Под популярностью понимается 
количество заведений этой сети в регионе. Постройте подходящую для такой информации визуализацию. Знакомы ли вам эти сети? 
Есть ли какой-то признак, который их объединяет? К какой категории заведений
они относятся

In [ ]:
chainmax=data.pivot_table(index='name',values='chain',aggfunc='sum')# sum так как у сетевых chain=1
chainmax.columns=['quantity']
chainmax=chainmax.reset_index()
chainmax=chainmax.sort_values(by='quantity',ascending=False)

In [ ]:
top15=chainmax.head(15)
top15

In [ ]:
fig5=px.bar(top15,x='quantity',y='name',orientation='h')
fig5.update_layout(title='Самые популярные сетевые заведения',xaxis_title='кол-во заведений популярной сети',yaxis_title='названия сети')
fig5.show()

Практически все заведения знакомы, в основном это кофейни, пиццерии и булочные средней ценовой категории

In [ ]:
chainmax1=data.pivot_table(index=['name','category'],values='chain',aggfunc='sum')# sum так как у сетевых chain=1
chainmax1=chainmax1.reset_index()

chainmax1.columns=['name','category','quantity']
chainmax1=chainmax1.sort_values(by='quantity',ascending=False).head(15)
display(chainmax1)

In [ ]:
categorytop15=chainmax1.groupby('category')['quantity'].sum().reset_index().sort_values(by='quantity',ascending=False)
categorytop15['percent']=100*categorytop15['quantity']/(categorytop15['quantity'].sum())
display(categorytop15)

In [ ]:
# check
x = data.copy()
df_chain = x[x['chain'] == 1] 
 
(
    df_chain
    .groupby(['name', 'category']).agg(count_name=('name', 'count')).sort_values(by='count_name').tail(15)
    .reset_index()
    .groupby('category')['count_name'].sum()
    .to_frame()
    .assign(percent_total=lambda x: x / x.sum()).round(2)
    .sort_values(by='percent_total')
)

In [ ]:
with plt.style.context('seaborn'):
    plt.bar(categorytop15['category'],categorytop15['percent'])
    plt.xticks(rotation=45)
    plt.title('% категорий популярных сетей')
    plt.ylabel('доля')
    plt.xlabel('Категории заведений')
    plt.show()

Таким образом, среди самых популярных сетевых заведений больше всего кофеен и ресторанов

### 3.5
Какие административные районы Москвы присутствуют в датасете? Отобразите общее количество заведений и количество 
заведений каждой категории по районам. Попробуйте проиллюстрировать эту информацию одним графиком

#### 3.5.1 Представлены следующие административные районы:

In [ ]:
data['district'].unique()

In [ ]:
pt1=data.pivot_table(index='district',columns='category',values='name',aggfunc='count')
pt1

In [ ]:
pt2=data.pivot_table(index=['district','category'],values='name',aggfunc='count')
pt2=pt2.sort_values(by='name',ascending=False).reset_index()
display(pt2)

#### 3.5.2 Категории заведений по округам

In [ ]:
fig5=px.bar(pt2,x='name',y='district',color='category',orientation='h')
fig5.update_layout(title='Категории заведений по округам',xaxis_title='кол-во заведений',yaxis={'categoryorder':'total descending'},yaxis_title='округ')
#,yaxis={'categoryorder':'total descending'},yaxis_title='округ'
fig5.show()

Таким образом, лидером по количеству заведений является центральный округ, при этом процентное соотношение категорий имеет 
много общего для каждого района

### 3.6
Визуализируйте распределение средних рейтингов по категориям заведений. 
Сильно ли различаются усреднённые рейтинги в разных типах общепита?

In [ ]:
pt3=data.pivot_table(index='category',values='rating',aggfunc='mean')
pt3=pt3.sort_values(by='rating',ascending=False).reset_index()
pt3.columns=['category','mean_rating']


display(pt3)

Средние рейтинги не сильно отличаются друг от друга в зависимости от категорий. Самый низкий рейтинг у заведений быстрого питания

In [ ]:
px.bar(pt3,x='category',y='mean_rating')

#Средний рейтинг по категориям

In [ ]:
fg=px.bar(pt3,x='category',y='mean_rating')
fg.update_layout(yaxis_range=[4,4.5])
fg.update_layout(title='Средний рейтинг заведений',xaxis_title='категории',yaxis_title='средний рейтинг')
fg.show()

Таким образом, разница в рейтинге у различных категорий отличается не сильно. При этом выше всего средний рейтинг у баров

### 3.7
Постройте фоновую картограмму (хороплет) со средним рейтингом заведений каждого района.
Границы районов Москвы, которые встречаются в датасете.

In [ ]:

import json


try:
    with open('/datasets/admin_level_geomap.geojson', 'r') as f:
        geo_json = json.load(f)
except:
    with open(r"C:\Users\Владислав\Documents\dat_an\admin_level_geomap.geojson") as f:
        geo_json = json.load(f)


print(json.dumps(geo_json, indent=2, ensure_ascii=False, sort_keys=True))

In [ ]:
pt4=data.pivot_table(index='district',values='rating',aggfunc='mean')
pt4=pt4.reset_index()
pt4.columns=['district','rating']
display(pt4)

In [ ]:
# импортируем карту и хороплет
from folium import Map, Choropleth

# загружаем JSON-файл с границами округов Москвы
#state_geo = r"C:\Users\Владислав\Documents\dat_an\admin_level_geomap.geojson"#выдает ошибку
state_geo = 'https://code.s3.yandex.net/data-analyst/admin_level_geomap.geojson'
# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10, tiles='Cartodb Positron')

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=pt4,
    columns=['district', 'rating'],
    key_on='feature.name',
    fill_color='YlGn',
    fill_opacity=0.6,
    legend_name='Средний рейтинг заведений по районам',
).add_to(m)

# выводим карту
m

В заведениях центрального округа средний рейтинг самый высокий

#### 3.8
Отобразите все заведения датасета на карте с помощью кластеров средствами библиотеки folium

In [ ]:
# импортируем карту и маркер
from folium import Map, Marker
# импортируем кластер
from folium.plugins import MarkerCluster

# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10, tiles="Cartodb Positron")
# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(m)

# пишем функцию, которая принимает строку датафрейма,
# создаёт маркер в текущей точке и добавляет его в кластер marker_cluster
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)

# применяем функцию create_clusters() к каждой строке датафрейма
data.apply(create_clusters, axis=1)

# выводим карту
m

<div class="alert alert-info">
Таким образом, на карту были нанесены кластера, при приближении к которым можно увидеть отдельные метки. Кластеры также 
позволяют увидеть количество объектов в районах, а также, при приближении карты и их расположение, при этом
для каждого объекта доступна информация о его названии и рейтинге. По результатам построения хороплета визуально заметна 
разница средних чеков по районам

#### 3.9
Найдите топ-15 улиц по количеству заведений. Постройте график распределения количества заведений и их категорий по этим улицам.
Попробуйте проиллюстрировать эту информацию одним графиком.

In [ ]:
pt5=data.pivot_table(index='street',values='name',aggfunc='count').rename(columns={'name':'name_count'})
pt5=pt5.sort_values(by='name_count',ascending=False).reset_index().head(15)

In [ ]:

display(pt5)

In [ ]:
pt6 = data.loc[data['street'].isin(pt5['street'])].pivot_table(index=['street', 'category'], values='name', aggfunc='count').reset_index()
pt6=pt6.sort_values(by='name',ascending=False)                     
display(pt6)                      


In [ ]:

#pt6=data.pivot_table(index=['street','category'],values='name',aggfunc='count').reset_index().sort_values(by='name',ascending=False)
#pt6.columns=['street','category','quantity']
#display(pt6.head(15))

In [ ]:
fig7=px.bar(pt6,x='name',y='street',color='category',orientation='h')
fig7.update_layout(title='Заведения на популярных улицах',xaxis_title='названия улиц',yaxis={'categoryorder':'total descending'},yaxis_title='кол-во заведений')
#,yaxis={'categoryorder':'total descending'},yaxis_title='округ'
fig7.show()

Таким образом, на популярных улицах в основном кафе и рестораны

#### 3.10
Найдите улицы, на которых находится только один объект общепита. Что можно сказать об этих заведениях?

In [ ]:

#onlyone=pt6.query('quantity==1')
#display(onlyone)

In [ ]:
#onlyone['category'].value_counts()

In [ ]:
pt10=data.pivot_table(index=['district','street'],values='name',aggfunc='count')
pt10=pt10.reset_index()
pt10=pt10.query('name==1')
pt10=pt10.pivot_table(index=['district'],values='name',aggfunc='count').reset_index()
pt10.columns=['district','quantity']
pt2new=pt2.pivot_table(index='district',values='name',aggfunc='sum').reset_index()
pt10['ratio']=pt10['quantity']/(pt2new['name'])

display(pt10)

In [ ]:
pt2new=pt2.pivot_table(index='district',values='name',aggfunc='sum').reset_index()
pt2new['name']

In [ ]:
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10, tiles='Cartodb Positron')

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=pt10,
    columns=['district', 'ratio'],
    key_on='feature.name',
    fill_color='YlGn',
    fill_opacity=0.6,
    legend_name='Средний рейтинг заведений по районам',
).add_to(m)

# выводим карту
m

<div class="alert alert-info">
Таким образом, в центральном округе больше всего (и в процентном соотношении) улиц с всего одним заведением, стоит тоже на них обратить внимание, возможно, 
рядом начали строить большой торговый центр или транспортный узел и данное место будет пользоваться бОльшим спросом

#### 3.11
Значения средних чеков заведений хранятся в столбце middle_avg_bill. Эти числа показывают примерную стоимость заказа в 
рублях, которая чаще всего выражена диапазоном. Посчитайте медиану этого столбца для каждого района. Используйте это 
значение в качестве ценового индикатора района. Постройте фоновую картограмму (хороплет) с полученными значениями для 
каждого района. Проанализируйте цены в центральном административном округе и других. Как удалённость от центра влияет на 
цены в заведениях?

In [ ]:
data['middle_avg_bill']

In [ ]:
pt7=data.pivot_table(index='district',values='middle_avg_bill',aggfunc='median').reset_index()
display(pt7)


In [ ]:
# moscow_lat - широта центра Москвы, moscow_lng - долгота центра Москвы
moscow_lat, moscow_lng = 55.751244, 37.618423

# создаём карту Москвы
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10, tiles='Cartodb Positron')

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=pt7,
    columns=['district', 'middle_avg_bill'],
    key_on='feature.name',
    fill_color='YlGn',
    fill_opacity=0.6,
    legend_name='Значение среднего чека по округам',
).add_to(m)

# выводим карту
m

В центре, что и следовало ожидать, цены самые большие. При этом удаленность от центра не сильно влияет
на ценовую категорию, влияет скорее сам округ и средняя стоимость жилья в нем

### 4 Анализ рынка кофеен

#### 4.1
Сколько всего кофеен в датасете? В каких районах их больше всего, каковы особенности их расположения?

In [ ]:

datacafe=data.query('category=="кофейня"')
datacafe.head()

In [ ]:
data.loc[data['category']=='кофейня','name'].count()

In [ ]:
pt8=datacafe.pivot_table(index='district',values='name',aggfunc='count').sort_values(by='name',ascending=False).reset_index()
display(pt8)

Всего 1404 кафейни и больше всего их  в центральном округе

#### 4.2
Есть ли круглосуточные кофейни? построить график

In [ ]:
datacafe.loc[data['is_24/7']==True,'name'].count() 

In [ ]:
pt11=datacafe.pivot_table(index=['district','is_24/7'],values='name',aggfunc='count').sort_values(by='name',ascending=False).reset_index()
#pt11.columns=['district','nonchain']
#pt11['total']=pt8['name']
display(pt11)

In [ ]:
fig10=px.bar(pt11,x='name',y='district',color='is_24/7',orientation='h')
fig10.update_layout(title='Режим работы кофеен по округам',xaxis_title='кол-во кофеен',yaxis_title='округ')

fig10.update_layout(legend_title_text='Круглосуточная работа')

fig10.show()



Круглосуточных кофеен очень мало, а где-то и совсем практически нет

#### 4.3
Какие у кофеен рейтинги? Как они распределяются по районам?

In [ ]:
datacafe['rating'].describe()

In [ ]:
pt9=datacafe.pivot_table(index='district',values='rating',aggfunc=['mean','median']).reset_index()
pt9.columns=['district','mean_rating','median_rating']
display(pt9)

In [ ]:
m2 = Map(location=[moscow_lat, moscow_lng], zoom_start=10, tiles='Cartodb Positron')

# создаём хороплет с помощью конструктора Choropleth и добавляем его на карту
Choropleth(
    geo_data=state_geo,
    data=pt9,
    columns=['district', 'mean_rating'],
    key_on='feature.name',
    fill_color='YlGn',
    fill_opacity=0.6,
    legend_name='Значение среднего рейтинга по округам',
).add_to(m2)

# выводим карту
m2

Медианный рейтинг практически везде одинаков, при этом по среднему рейтингу опережает всех центральный округ

#### 4.4
На какую стоимость чашки капучино стоит ориентироваться при открытии и почему?

In [ ]:
datacafe['middle_coffee_cup'].median()

In [ ]:
pt10=datacafe.pivot_table(index='district',values='middle_coffee_cup',aggfunc='median').reset_index().sort_values(by='middle_coffee_cup',ascending=False)
display(pt10)

In [ ]:
datacafe['middle_coffee_cup'].corr(datacafe['rating'])

In [ ]:
datacafe.plot(x='rating',y='middle_coffee_cup',kind='scatter')

#### 4.5 Проведем дополнительный анализ

Взаимосвязь между  рейтингом и средним счетом незначительная. Я думаю, для начала стоит ориентироваться 
на медианную стоимость в 170 рублей за чашку кофе

In [ ]:
datacafe['middle_avg_bill'].corr(datacafe['rating'])

In [ ]:
from sklearn.neighbors import KernelDensity
kde=KernelDensity(kernel='gaussian',bandwidth=0.5).fit(np.array(data['rating']).reshape(1, -1))
kde.get_params()

In [ ]:
from seaborn import kdeplot

Визуализируем оценку плотности распределения рейтинга

In [ ]:
my_kde=kdeplot(data['rating'])

Попробуем понять, можно ли сказать, что рейтинг распределен нормально, с соответствующей выборочной оценкой
мат.ожидания и дисперсии.

In [ ]:
mu=data['rating'].mean()


In [ ]:
sigma=data['rating'].std()


Произведем Z-нормализацию

In [ ]:
normeddata=(data['rating'].to_numpy()-mu)/sigma

In [ ]:
from scipy import stats

Воспользуемся критерием согласия Колмогорова

In [ ]:
stats.kstest(normeddata,'norm')

Таким образом, нельзя сделать вывод о равенстве распределений

In [ ]:
plt.hist(normeddata,bins=20);
plt.hist(stats.norm(loc=0,scale=1).rvs(size=8171),bins=20);
plt.title('Сравнение нормального распределения с выборочными данными рейтинга')
plt.ylabel('кол-во заведений')


Для дальнейших оценок могут использоваться более сложные методы, а также оценка плотности, например KDE
или с использованием различных полиномов и дальнейшего сравнения

Теперь попробуем понять, можно ли на основе данных сделать вывод о том, какой рейтинг получит кофейня
после открытия. Поскольку рейтинг очень важен для пользователей, особенно на первом этапе, когда кофейня
не получила еще известность и нет постоянных клиентов

In [ ]:
#middle_coffee_cup
pt12=data.pivot_table(index='street',values='middle_coffee_cup',aggfunc='mean').reset_index().sort_values(by='middle_coffee_cup',ascending=False).head(5)
display(pt12)

In [ ]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

In [ ]:
datacafe.info()

In [ ]:
datacafe['price'].unique()

In [ ]:
distunique=datacafe['district'].unique()
distunique

In [ ]:
def chain(datacafe):
    if datacafe['chain']==True:
        return 1
    else:
        return 0
    
def workinghourscafe(datacafe):
    #hours=datacafe['hours']
    if datacafe['hours']=='ежедневно, круглосуточно':
        return 1
    else:
        return 0
def distr(datacafe):
    #hours=datacafe['hours']

    pr=datacafe['district']
    if pr==distunique[0]:
        return 0
    elif pr==distunique[1]:
        return 1
    elif pr==distunique[2]:
        return 2
    elif pr==distunique[3]:
        return 3
    elif pr==distunique[4]:
        return 4
    elif pr==distunique[5]:
        return 5
    elif pr==distunique[6]:
        return 6
    elif pr==distunique[7]:
        return 7
    else:
        return 8
   
def pricecategory(datacafe):
    pr=datacafe['price']
    if pr=='низкие':
        return 0
    elif pr=='средние':
        return 1
    elif pr=='выше среднего':
        return 2
    elif pr=='высокие':
        return 3


In [ ]:

newdc=datacafe.loc[:,['price','seats','rating']]


newdc['chain']=datacafe.apply(chain,axis=1)
newdc['24/7']=datacafe.apply(workinghourscafe,axis=1)
newdc['price']=datacafe.apply(pricecategory,axis=1)
newdc['district']=datacafe.apply(distr,axis=1)
newdc=newdc.dropna()

Таким образом, категориальные данные перевели в формат чисел

In [ ]:
newdc.info()

In [ ]:
newdc.head()

In [ ]:
newdc=newdc.dropna()

In [ ]:
y=newdc['rating'].apply(round)
#y

In [ ]:
X=newdc.loc[:,['price','seats','chain','24/7','district']]
display(X)

In [ ]:
from sklearn.model_selection import GridSearchCV

Произведем подбор лучших параметров, для простоты в качестве критерия оставим только энтропию Шеннона

In [ ]:

m=DecisionTreeClassifier()
#parametrs = {'criterion': ['gini', 'entropy'], 'max_depth': range(3, 8), 'min_samples_split': range(2, 10), 'min_samples_leaf': range(1, 10)}
parametrs = {'criterion': ['entropy'], 'max_depth': range(2, 8), 'min_samples_split': range(2, 10), 
             'min_samples_leaf': range(1, 10)}
gscv=GridSearchCV(m,parametrs,cv=5,n_jobs=-1)
gscv.fit(X,y)

In [ ]:
gscv.best_params_

Визуализируем основные признаки, влияющие на классификацию

In [ ]:

mm=DecisionTreeClassifier(criterion='entropy',max_depth=3,min_samples_leaf=1,min_samples_split=2)
mm.fit(X, y)
imp = pd.DataFrame(mm.feature_importances_, index=X.columns, columns=['importance'])
imp.sort_values('importance').plot(kind='barh', figsize=(12, 8))
plt.title('Основные признаки')
plt.ylabel('названия столбцов')
plt.xlabel('важность')
plt.show()

In [ ]:
newdc['district'].unique()

In [ ]:
newdc['rating'].apply(round).value_counts()# посмотрим, что изначально находится в value и на какой позиции, и как это дальше 
#разбивается
#value=[3rating,4rating,5rating]

In [ ]:
X;

Построим дерево решений

In [ ]:
tree.plot_tree(mm)

Не сетевая кофейня со средними ценами, и работающая круглосуточно, привлечет скорее всего много пользователей
и получит высокий рейтинг

In [ ]:
print('предполагаемый рейтинг: ',mm.predict(np.array([1,4,0,1,3]).reshape(1,-1))[0])
#mm.predict(np.array([1,4,0,1,3]).reshape(1,-1))[0]#несетевая мало мест недорогая круглосуточно

Сетевая кофейня с высокими ценами, работающая только в определенное время, получит меньше рейтинг и 
будет не так популярна

In [ ]:
print('предполагаемый рейтинг: ',mm.predict(np.array([3,4,1,0,3]).reshape(1,-1))[0])#дорогая сетевая

Однозначный вывод сложно сделать ввиду небольшого количества данных. Для дальнейшего анализа аналитик
может по-другому подобрать признаки, для получения более репрезентативной выборки.
При этом стоит для начала решить, будет ли кофейня сетевой или нет, внимательно подойти к выбору
площади заведения и району расположения. Индивидуальность заведения обычно увеличивает его престиж, 
при этом сетевые кафе с высокими ценами могут отпугнуть посетителей.

In [ ]:
from sklearn.manifold import TSNE

### блок обновления библиотек

In [ ]:
# pip install numpy --upgrade

In [ ]:
# pip install scikit-learn --upgrade

In [ ]:
import numpy as np

In [ ]:
# import numpy
# print(numpy.__version__)

In [ ]:
# import sklearn
# print(sklearn.__version__)

In [ ]:
XX=TSNE(n_components=2,learning_rate='auto',init='random').fit_transform(newdc)

Воспользуемся методом понижения размерности tSNE, основанным на расстоянии Кульбака-Лейблера, для дальнейшей визуализации, чтобы понять  разделяется ли
данная выборка на каие-то группы явно или последующая кластеризация не является тривиальной задачей

In [ ]:
plt.scatter(XX[:,0],XX[:,1])

Четкие кластеры сложно выделить на первый взгляд

### 5 Заключение и общий вывод

**Заключение**
В рамках работы был проанализирован рынок общественного питания Москвы. В результате анализа 
можно сделать следующие выводы:
    1) Самые популярные заведения - кафе и рестораны;\
    2) больше всего мест в барах и ресторанах;\
    3) сетевые заведения представлены реже, чем не сетевые, при этом среди сетевых обычно пиццерии и булочные;\
    4) больше всего заведений, что ожидаемо, находится в центральном округу, при этом в каждом округу;\
    самые популярные заведения - кофейни и рестораны;\
    5) средние рейтинги не сильно отличаются по категориям, при этом рейтинги и средний чек зависят 
    от района.

В рамках анализа кофеен можно сделать следующие выводы:\
    1) Всего представлено 1404 кофейни, придется столкнуться с большой конкуренцией;\
    2) самая высокая конкуренция в центральном и северном округах;\
    3) круглосуточных кофеен очень мало;\
    4) самый высокий рейтинг в среднем в центральном округе;
    5) Сложно сделать однозначные выводы, требуется дополнительная информация о доходах кофеен, а ценах
    аренды помещений, информация о закрывшихся кофейнях в связи с убыточностью. 

При открытии кофейни первым делом следует определить: есть ли планы делать ее сетевой, выбрать формат,
    вместимость и район.
    Я бы предложил сделать круглосуточную кофейню со среднем ценовым диапазоном и расположить ее 
    в центре если нет страха перед конкуренцией, либо в одном из не самых популярных районов, тогда
    ввиду своей доступности, ценам и режиму работы, кофейня будет пользоваться спросом, скорее всего 
    получит высокий рейтинг и будет приносить прибыль

В среднем, дороже всего кофе стоит на следующих улицах
 (большая семёновская улица,большая никитская улица,3-я фрунзенская улица,большая сухаревская площадь,богословский переулок), при этом, ввиду 
небольшого количества даных по среднему значению не всегда стоит ориентироваться в данном случае



**Ссылка на презентацию (исправленная)** 
https://cloud.mail.ru/public/zkX5/1w9y4hKYh